V.2

-deleted prediction year from train

V.3

-add kenpom

-add 538 team rating

In [ ]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMRegressor
from lightgbm import LGBMClassifier
import lightgbm
import matplotlib.pyplot as plt 
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import log_loss
import warnings
warnings.filterwarnings('ignore')

Load kenpom

In [ ]:
NCAA = pd.read_csv('../input/kenpom-2020/NCAA2021_Kenpom.csv')
NCAA['record_w'] = NCAA['record'].apply(lambda x: int(x.split('-')[0]))
NCAA['record_l'] = NCAA['record'].apply(lambda x: int(x.split('-')[1]))
NCAA = NCAA.drop(['FirstD1Season','TeamName','FirstD1Season','LastD1Season','team','Seed','conference','record'],axis = 1)
NCAA.head()

In [ ]:
# Kenpom = pd.read_csv('../input/kenpom-2020/Mkenpom2021.csv')
# Teams = pd.read_csv('../input/ncaam-march-mania-2021/MDataFiles_Stage2/MTeamSpellings.csv',encoding='cp1251')
# Kenpom['Team'] = Kenpom['Team'].str.lower()
# Kenpom = Kenpom.merge(Teams, how='left', left_on=['Team'], right_on=['TeamNameSpelling'])
# Kenpom = Kenpom.dropna(subset=['TeamID'])
# Kenpom['TeamID'] = Kenpom['TeamID'].astype(int)
# Kenpom = Kenpom[['Year','TeamID','Wins','Losses','Pyth']]
# Kenpom.head()

load 538 team ratings

In [ ]:
men538 = pd.read_csv('../input/ncaa-men-538-team-ratings/538ratingsMen.csv').drop(['TeamName'],axis = 1)
men538.head()

# X prepare

In [ ]:
MNCAATourneyDetailedResults = pd.read_csv('../input/ncaam-march-mania-2021/MDataFiles_Stage2/MNCAATourneyDetailedResults.csv')
MNCAATourneyCompactResults = pd.read_csv('../input/ncaam-march-mania-2021/MDataFiles_Stage2/MNCAATourneyCompactResults.csv')
TourneyResults = MNCAATourneyDetailedResults.merge(MNCAATourneyCompactResults, on= MNCAATourneyCompactResults.columns.to_list(), how='right')
TourneyResults['TypeCompetition'] = 'Tourney'

MRegularSeasonDetailedResults = pd.read_csv('../input/ncaam-march-mania-2021/MDataFiles_Stage2/MRegularSeasonDetailedResults.csv')
MRegularSeasonCompactResults = pd.read_csv('../input/ncaam-march-mania-2021/MDataFiles_Stage2/MRegularSeasonCompactResults.csv')
SeasonResults = MRegularSeasonDetailedResults.merge(MRegularSeasonCompactResults, on= MRegularSeasonCompactResults.columns.to_list(), how='right')
SeasonResults['TypeCompetition'] = 'Season'

X = TourneyResults.merge(SeasonResults, on= SeasonResults.columns.to_list(), how='outer')

X = X.drop(['DayNum',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'], axis=1)
X.head()

shuffle teams

In [ ]:
A = X[X['WLoc']=='A'].rename(columns={
    'WTeamID': 'BTeamID',
    'WScore': 'BScore',
    'LTeamID': 'ATeamID',
    'LScore': 'AScore',
}).drop('WLoc', axis=1)

H = X[X['WLoc']=='H'].rename(columns={
    'WTeamID': 'ATeamID',
    'WScore': 'AScore',
    'LTeamID': 'BTeamID',
    'LScore': 'BScore',
}).drop('WLoc', axis=1)

N = X[X['WLoc']=='N'].rename(columns={
    'WTeamID': 'ATeamID',
    'WScore': 'AScore',
    'LTeamID': 'BTeamID',
    'LScore': 'BScore',
})

import random
for index in N.index:
    if random.randint(0, 1) == 1:
        N.at[index, 'WLoc'] = N.at[index, 'ATeamID']
        N.at[index, 'ATeamID'] = N.at[index, 'BTeamID']
        N.at[index, 'BTeamID'] = N.at[index, 'WLoc']
        
        N.at[index, 'WLoc'] = N.at[index, 'AScore']
        N.at[index, 'AScore'] = N.at[index, 'BScore']
        N.at[index, 'BScore'] = N.at[index, 'WLoc']

N = N.drop('WLoc', axis=1)  

X = H.append(A, ignore_index=False,sort=False)
X = X.append(N, ignore_index=False,sort=False)
X.head()

add kenpom and 538 team ratings to X

In [ ]:
X = X.merge(NCAA, how='left', left_on=['Season','ATeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)
X = X.merge(NCAA, how='left', left_on=['Season','BTeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)

# X = X.merge(Kenpom, how='left', left_on=['Season','ATeamID'], right_on=['Year','TeamID']).drop(['TeamID','Year'], axis=1)
# X = X.merge(Kenpom, how='left', left_on=['Season','BTeamID'], right_on=['Year','TeamID']).drop(['TeamID','Year'], axis=1)

X = X.merge(men538, how='left', left_on=['Season','ATeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)
X = X.merge(men538, how='left', left_on=['Season','BTeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)

X = X.dropna(subset=['rank_x','rank_y'])
X.head()

In [ ]:
X['rank'] = X['rank_x'] - X['rank_y']
X['adj_em'] = X['adj_em_x'] - X['adj_em_y']
X['adj_o'] = X['adj_o_x'] - X['adj_o_y']
X['adj_o_rank'] = X['adj_o_rank_x'] - X['adj_o_rank_y']
X['adj_d'] = X['adj_d_x'] - X['adj_d_y']
X['adj_d_rank'] = X['adj_d_rank_x'] - X['adj_d_rank_y']
X['adj_tempo'] = X['adj_tempo_x'] - X['adj_tempo_y']
X['adj_tempo_rank'] = X['adj_tempo_rank_x'] - X['adj_tempo_rank_y']
X['luck_rank'] = X['luck_rank_x'] - X['luck_rank_y']
X['sos_adj_em'] = X['sos_adj_em_x'] - X['sos_adj_em_y']
X['sos_adj_em_rank'] = X['sos_adj_em_rank_x'] - X['sos_adj_em_rank_y']
X['nc_sos_adj_em'] = X['nc_sos_adj_em_x'] - X['nc_sos_adj_em_y']
X['nc_sos_adj_em_rank'] = X['nc_sos_adj_em_rank_x'] - X['nc_sos_adj_em_rank_y']
X['ncaa_seed'] = X['ncaa_seed_x'] - X['ncaa_seed_y']
X['record_w'] = X['record_w_x'] - X['record_w_y']
X['record_l'] = X['record_l_x'] - X['record_l_y']

# X['Rank'] = X['Rank_x'] - X['Rank_y']
# X['Wins'] = X['Wins_x'] - X['Wins_y']
# X['Losses'] = X['Losses_x'] - X['Losses_y']
# X['Pyth'] = X['Pyth_x'] - X['Pyth_y']

X['538rating'] = X['538rating_x'] - X['538rating_y']

X = X.drop(['rank_x', 'adj_em_x', 'adj_o_x',
       'adj_o_rank_x', 'adj_d_x', 'adj_d_rank_x', 'adj_tempo_x',
       'adj_tempo_rank_x', 'luck_x', 'luck_rank_x', 'sos_adj_em_x',
       'sos_adj_em_rank_x', 'sos_adj_o_x', 'sos_adj_o_rank_x', 'sos_adj_d_x',
       'sos_adj_d_rank_x', 'nc_sos_adj_em_x', 'nc_sos_adj_em_rank_x',
       'ncaa_seed_x', 'record_w_x', 'record_l_x', 'rank_y', 'adj_em_y',
       'adj_o_y', 'adj_o_rank_y', 'adj_d_y', 'adj_d_rank_y', 'adj_tempo_y',
       'adj_tempo_rank_y', 'luck_y', 'luck_rank_y', 'sos_adj_em_y',
       'sos_adj_em_rank_y', 'sos_adj_o_y', 'sos_adj_o_rank_y', 'sos_adj_d_y',
       'sos_adj_d_rank_y', 'nc_sos_adj_em_y', 'nc_sos_adj_em_rank_y',
       'ncaa_seed_y', 'record_w_y', 'record_l_y',
       '538rating_x', '538rating_y'],axis = 1)

get y

In [ ]:
X['HomeWin'] = (X['AScore']-X['BScore'] > 0).astype(int)
X.head()

# test prepare

In [ ]:
test = pd.read_csv('../input/ncaam-march-mania-2021/MDataFiles_Stage2/MSampleSubmissionStage2.csv')
submission = pd.read_csv('../input/ncaam-march-mania-2021/MDataFiles_Stage2/MSampleSubmissionStage2.csv')

test['Season'] = test['ID'].apply(lambda x: int(x.split('_')[0]))
test['ATeamID'] = test['ID'].apply(lambda x: int(x.split('_')[1]))
test['BTeamID'] = test['ID'].apply(lambda x: int(x.split('_')[2]))
test = test.drop(['Pred','ID'], axis=1)
test['TypeCompetition'] = 'Tourney'
test.head()

add kenpom and 538 team ratings to test

In [ ]:
test = test.merge(NCAA, how='left', left_on=['Season','ATeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)
test = test.merge(NCAA, how='left', left_on=['Season','BTeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)

# test = test.merge(Kenpom, how='left', left_on=['Season','ATeamID'], right_on=['Year','TeamID']).drop(['TeamID','Year'], axis=1)
# test = test.merge(Kenpom, how='left', left_on=['Season','BTeamID'], right_on=['Year','TeamID']).drop(['TeamID','Year'], axis=1)

test = test.merge(men538, how='left', left_on=['Season','ATeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)
test = test.merge(men538, how='left', left_on=['Season','BTeamID'], right_on=['Season','TeamID']).drop(['TeamID'], axis=1)
test.head()

In [ ]:
test['rank'] = test['rank_x'] - test['rank_y']
test['adj_em'] = test['adj_em_x'] - test['adj_em_y']
test['adj_o'] = test['adj_o_x'] - test['adj_o_y']
test['adj_o_rank'] = test['adj_o_rank_x'] - test['adj_o_rank_y']
test['adj_d'] = test['adj_d_x'] - test['adj_d_y']
test['adj_d_rank'] = test['adj_d_rank_x'] - test['adj_d_rank_y']
test['adj_tempo'] = test['adj_tempo_x'] - test['adj_tempo_y']
test['adj_tempo_rank'] = test['adj_tempo_rank_x'] - test['adj_tempo_rank_y']
test['luck_rank'] = test['luck_rank_x'] - test['luck_rank_y']
test['sos_adj_em'] = test['sos_adj_em_x'] - test['sos_adj_em_y']
test['sos_adj_em_rank'] = test['sos_adj_em_rank_x'] - test['sos_adj_em_rank_y']
test['nc_sos_adj_em'] = test['nc_sos_adj_em_x'] - test['nc_sos_adj_em_y']
test['nc_sos_adj_em_rank'] = test['nc_sos_adj_em_rank_x'] - test['nc_sos_adj_em_rank_y']
test['ncaa_seed'] = test['ncaa_seed_x'] - test['ncaa_seed_y']
test['record_w'] = test['record_w_x'] - test['record_w_y']
test['record_l'] = test['record_l_x'] - test['record_l_y']

# test['Rank'] = test['Rank_x'] - test['Rank_y']
# test['Wins'] = test['Wins_x'] - test['Wins_y']
# test['Losses'] = test['Losses_x'] - test['Losses_y']
# test['Pyth'] = test['Pyth_x'] - test['Pyth_y']

test['538rating'] = test['538rating_x'] - test['538rating_y']

test = test.drop(['rank_x', 'adj_em_x', 'adj_o_x',
       'adj_o_rank_x', 'adj_d_x', 'adj_d_rank_x', 'adj_tempo_x',
       'adj_tempo_rank_x', 'luck_x', 'luck_rank_x', 'sos_adj_em_x',
       'sos_adj_em_rank_x', 'sos_adj_o_x', 'sos_adj_o_rank_x', 'sos_adj_d_x',
       'sos_adj_d_rank_x', 'nc_sos_adj_em_x', 'nc_sos_adj_em_rank_x',
       'ncaa_seed_x', 'record_w_x', 'record_l_x', 'rank_y', 'adj_em_y',
       'adj_o_y', 'adj_o_rank_y', 'adj_d_y', 'adj_d_rank_y', 'adj_tempo_y',
       'adj_tempo_rank_y', 'luck_y', 'luck_rank_y', 'sos_adj_em_y',
       'sos_adj_em_rank_y', 'sos_adj_o_y', 'sos_adj_o_rank_y', 'sos_adj_d_y',
       'sos_adj_d_rank_y', 'nc_sos_adj_em_y', 'nc_sos_adj_em_rank_y',
       'ncaa_seed_y', 'record_w_y', 'record_l_y',
       '538rating_x', '538rating_y'],axis = 1)

In [ ]:
temp = X.append(test, ignore_index=False,sort=False)
temp = pd.get_dummies(temp,dtype=bool)
X = temp[:len(X)]
test = temp[len(X):]
temp = pd.DataFrame
test = test.drop(['AScore','BScore','HomeWin'],axis = 1)

In [ ]:
test

In [ ]:
X

# Train

In [ ]:
lgbm_parameters= {
    'objective': 'binary',
    'metric': 'binary_logloss',
}

In [ ]:
test_pred = np.zeros(len(test))
test_pred = []

kf = KFold(n_splits=10, shuffle=True)

for year in test['Season'].unique():
    
    #X_year = X[(X['Season'] >= year-5)&(X['Season'] <= year+5)]
    X_year = X[((X['TypeCompetition_Season'] == True)&(X['Season'] == year))|(X['Season'] != year)]
    
    y_year = X_year['HomeWin']
    X_year = X_year.drop(['AScore','BScore','HomeWin'], axis=1)
    test_year = test[test['Season'] == year]
 
    lgbm_val_pred = np.zeros(len(y_year))
    lgbm_test_pred = np.zeros(len(test_year))
    logloss = []
    
    for trn_idx, val_idx in kf.split(X_year,y_year):
        x_train_idx = X_year.iloc[trn_idx]
        y_train_idx = y_year.iloc[trn_idx]
        x_valid_idx = X_year.iloc[val_idx]
        y_valid_idx = y_year.iloc[val_idx]

        lgbm_model = LGBMClassifier(**lgbm_parameters)
        lgbm_model.fit(x_train_idx, y_train_idx, eval_set = ((x_valid_idx,y_valid_idx)),verbose = False, early_stopping_rounds = 100,categorical_feature=[0,1,2])
        lgbm_test_pred += lgbm_model.predict_proba(test_year)[:,1]/10
        logloss.append(log_loss(y_valid_idx, lgbm_model.predict_proba(x_valid_idx)[:,1])) 
        
    test_pred += lgbm_test_pred.tolist()
    
    print('Year_Predict:',year,'Log_Loss:',np.mean(logloss))

get submission

In [ ]:
submission.Pred = test_pred  
submission.to_csv('submission.csv', index=False)

Feature importance

In [ ]:
plt.rcParams["figure.figsize"] = (5, 10)
lightgbm.plot_importance(lgbm_model,height=.9)